In [ ]:
import requests
import pandas as pd
import numpy as np
from keys import *

#Nombre de la Ciudad 
city = "Florence"

#Abreviatura de del PAIS
country = "IT"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

In [ ]:
data = response.json()
#print(data)

# Extract forecast list
forecast_list = data.get('list', [])

# Prepare lists for DataFrame columns
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

# Create DataFrame
df = pd.DataFrame({
    'time': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind_speed': wind_speeds,
    'rain_volume_3h': rain_volumes,
    'snow_volume_3h': snow_volumes
})

print(df.head())

{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1758758400, 'main': {'temp': 16.21, 'feels_like': 15.9, 'temp_min': 16.21, 'temp_max': 16.21, 'pressure': 1016, 'sea_level': 1016, 'grnd_level': 993, 'humidity': 77, 'temp_kf': 0}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'clouds': {'all': 61}, 'wind': {'speed': 1.6, 'deg': 166, 'gust': 3.93}, 'visibility': 10000, 'pop': 0.2, 'rain': {'3h': 0.19}, 'sys': {'pod': 'n'}, 'dt_txt': '2025-09-25 00:00:00'}, {'dt': 1758769200, 'main': {'temp': 16.31, 'feels_like': 16.03, 'temp_min': 16.31, 'temp_max': 16.51, 'pressure': 1016, 'sea_level': 1016, 'grnd_level': 992, 'humidity': 78, 'temp_kf': -0.2}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'clouds': {'all': 64}, 'wind': {'speed': 1.98, 'deg': 144, 'gust': 3.4}, 'visibility': 10000, 'pop': 0.2, 'rain': {'3h': 0.13}, 'sys': {'pod': 'n'}, 'dt_txt': '2025-09-25 03:00:00'}, {'dt': 1758780000, 'main': {'temp